In [1]:
 pip install -Uq langgraph langsmith langchain_groq langchain_tavily

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.3/386.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 6.6 MB/s eta 0:00:00


In [2]:
from os import environ
from google.colab import userdata

environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')
environ["LANGSMITH_API_KEY"] = userdata.get('LANGSMITH_API_KEY')

environ["LANGCHAIN_TRACING_V2"] = "true"
environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
environ["LANGCHAIN_PROJECT"] = "05-LangGraph-Streaming-Outputs"

In [3]:
from langchain_groq import ChatGroq
from langchain_tavily import TavilySearch
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
from langchain_core.messages import ChatMessage
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables import RunnableConfig
from langgraph.graph import StateGraph
from IPython.display import Image, display

search = TavilySearch()
tools = [search]

llm = ChatGroq(model="openai/gpt-oss-20b")
llm_with_tools = llm.bind_tools(tools)

class State(TypedDict):
  messages: Annotated[list[ChatMessage], add_messages]


memory = MemorySaver()


def chatbot(state: State):
  answer = llm_with_tools.invoke(state["messages"])
  return {"messages": [answer]}


tool_node = ToolNode(tools)

graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)
graph_builder.add_conditional_edges("chatbot", tools_condition)
graph_builder.add_edge("tools", "chatbot")
graph_builder.set_entry_point("chatbot")
graph = graph_builder.compile(checkpointer=memory)

#display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
list(graph.channels)

['messages',
 '__start__',
 '__pregel_tasks',
 'branch:to:chatbot',
 'branch:to:tools']

In [ ]:
config = RunnableConfig(recursion_limit=10, configurable={"thread_id": "1"});


input = {"messages": [("user", "랭그래프에 대해서 인터넷 검색해서 알려주세요")]}


for event in graph.stream(input, config=config, interrupt_before=["tools"]):
  for key, value in event.items():
    print(f"{key}: {value}")

chatbot: {'messages': [AIMessage(content='', additional_kwargs={'reasoning_content': 'The user wants information about "랭그래프" (Korean). Likely means "랭킹 그래프" maybe? But "랭그래프" might be a specific term? Could be "랭크 그래프" or "랭크 그래프" meaning rank graph. But the user wrote "랭그래프" as one word. Let\'s search.', 'tool_calls': [{'id': 'fc_a24ff2fb-b163-4e2e-bbed-f1b23c6dd154', 'function': {'arguments': '{"query":"랭그래프","search_depth":"advanced"}', 'name': 'tavily_search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 113, 'prompt_tokens': 1367, 'total_tokens': 1480, 'completion_time': 0.100996795, 'prompt_time': 0.104199434, 'queue_time': 0.017993947, 'total_time': 0.205196229}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_6d85dbaad9', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--f7153d54-bf15-4fda-a15d-76dae24370ca-0', tool_calls=[{'name': 'tavily_search', 'args': {'query': '랭그래프', 'search_depth': '

In [ ]:
snapshot = graph.get_state(config)
snapshot.next

('tools',)

In [ ]:
snapshot.values["messages"][-1].tool_calls

[{'name': 'tavily_search',
  'args': {'query': '랭그래프', 'search_depth': 'advanced'},
  'id': 'fc_a24ff2fb-b163-4e2e-bbed-f1b23c6dd154',
  'type': 'tool_call'}]

In [ ]:
last_message = snapshot.values["messages"][-1]
last_message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  tavily_search (fc_a24ff2fb-b163-4e2e-bbed-f1b23c6dd154)
 Call ID: fc_a24ff2fb-b163-4e2e-bbed-f1b23c6dd154
  Args:
    query: 랭그래프
    search_depth: advanced


In [ ]:
# 수정하고자 하는 `ToolMessage` 의 `tool_call_id` 추출
tool_name = last_message.tool_calls[0]["name"]
tool_call_id = last_message.tool_calls[0]["id"]
print(tool_call_id)

fc_a24ff2fb-b163-4e2e-bbed-f1b23c6dd154


In [ ]:
graph.update_state

<bound method Pregel.update_state of <langgraph.graph.state.CompiledStateGraph object at 0x797c69c4c380>>

In [ ]:
ToolMessage

langchain_core.messages.tool.ToolMessage

In [ ]:
from langchain_core.messages import AIMessage, ToolMessage

modified_search_result = """[수정된 웹 검색 결과]
LangGraph는 상태 기반의 다중 액터 애플리케이션을 LLM을 활용해 구축할 수 있도록 지원합니다.
LangGraph는 사이클 흐름, 제어 가능성, 지속성, 클라우드 배포 기능을 제공하는 오픈 소스 라이브러리입니다.

자세한 튜토리얼은 [LangGraph 튜토리얼](https://langchain-ai.github.io/langgraph/tutorials/) 과
테디노트의 [랭체인 한국어 튜토리얼](https://wikidocs.net/233785) 을 참고하세요."""

new_messages = [
    ToolMessage(
        tool_call_id=tool_call_id,
        name=tool_name,
        content=modified_search_result,
    ),
]

new_messages[-1].pretty_print()


graph.update_state(
    config,
    {"messages": new_messages},
    as_node="tools",
)

print("(최근 1개의 메시지 출력)\n")
print(graph.get_state(config).values["messages"][-1])

================================= Tool Message =================================
Name: tavily_search

[수정된 웹 검색 결과] 
LangGraph는 상태 기반의 다중 액터 애플리케이션을 LLM을 활용해 구축할 수 있도록 지원합니다.
LangGraph는 사이클 흐름, 제어 가능성, 지속성, 클라우드 배포 기능을 제공하는 오픈 소스 라이브러리입니다.

자세한 튜토리얼은 [LangGraph 튜토리얼](https://langchain-ai.github.io/langgraph/tutorials/) 과
테디노트의 [랭체인 한국어 튜토리얼](https://wikidocs.net/233785) 을 참고하세요.
(최근 1개의 메시지 출력)

content='[수정된 웹 검색 결과] \nLangGraph는 상태 기반의 다중 액터 애플리케이션을 LLM을 활용해 구축할 수 있도록 지원합니다.\nLangGraph는 사이클 흐름, 제어 가능성, 지속성, 클라우드 배포 기능을 제공하는 오픈 소스 라이브러리입니다.\n\n자세한 튜토리얼은 [LangGraph 튜토리얼](https://langchain-ai.github.io/langgraph/tutorials/) 과\n테디노트의 [랭체인 한국어 튜토리얼](https://wikidocs.net/233785) 을 참고하세요.' name='tavily_search' id='5d9f766f-16d1-4802-9f88-78e31a281aa8' tool_call_id='fc_a24ff2fb-b163-4e2e-bbed-f1b23c6dd154'


In [ ]:
snapshot = graph.get_state(config)
snapshot.next

('chatbot',)

In [ ]:
for event in graph.stream(None, config, stream_mode="values"):
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================= Tool Message =================================
Name: tavily_search

[수정된 웹 검색 결과] 
LangGraph는 상태 기반의 다중 액터 애플리케이션을 LLM을 활용해 구축할 수 있도록 지원합니다.
LangGraph는 사이클 흐름, 제어 가능성, 지속성, 클라우드 배포 기능을 제공하는 오픈 소스 라이브러리입니다.

자세한 튜토리얼은 [LangGraph 튜토리얼](https://langchain-ai.github.io/langgraph/tutorials/) 과
테디노트의 [랭체인 한국어 튜토리얼](https://wikidocs.net/233785) 을 참고하세요.
================================== Ai Message ==================================

**랭그래프(LangGraph)란?**  
랭그래프는 LLM(대형 언어 모델)을 활용해 **상태 기반(state‑based) 다중 액터(multi‑actor) 애플리케이션**을 쉽고 효율적으로 구축할 수 있게 해 주는 오픈소스 라이브러리입니다.  
주요 목표는 다음과 같습니다.

| 목표 | 설명 |
|------|------|
| **사이클 흐름 관리** | 복잡한 워크플로우를 순차·병렬로 설계하고, 각 단계별 상태를 명확히 정의 |
| **컨트롤러(컨트롤러) 지원** | 실행 흐름을 동적으로 제어하고, 필요 시 중단·재개·분기 가능 |
| **지속성** | 실행 중인 상태를 저장하고, 재시작 시 복원 가능 |
| **클라우드 배포** | Docker, Kubernetes, Serverless 등 다양한 환경에 손쉽게 배포 |

---

### 핵심 개념

1. **State (상태)**  
   - 각 액터(Agent)가 현재 상황을 담은 객체.  
   - LLM의 입력/출력, 메타데이터, 외부 API 호출 결과 등을 포함.

2. **No

In [ ]:
snapshot = graph.get_state(config)

for state in snapshot.values["messages"]:
  state.pretty_print()

================================ Human Message =================================

랭그래프에 대해서 인터넷 검색해서 알려주세요
================================== Ai Message ==================================
Tool Calls:
  tavily_search (fc_a24ff2fb-b163-4e2e-bbed-f1b23c6dd154)
 Call ID: fc_a24ff2fb-b163-4e2e-bbed-f1b23c6dd154
  Args:
    query: 랭그래프
    search_depth: advanced
================================= Tool Message =================================
Name: tavily_search

[수정된 웹 검색 결과] 
LangGraph는 상태 기반의 다중 액터 애플리케이션을 LLM을 활용해 구축할 수 있도록 지원합니다.
LangGraph는 사이클 흐름, 제어 가능성, 지속성, 클라우드 배포 기능을 제공하는 오픈 소스 라이브러리입니다.

자세한 튜토리얼은 [LangGraph 튜토리얼](https://langchain-ai.github.io/langgraph/tutorials/) 과
테디노트의 [랭체인 한국어 튜토리얼](https://wikidocs.net/233785) 을 참고하세요.
================================== Ai Message ==================================

**랭그래프(LangGraph)란?**  
랭그래프는 LLM(대형 언어 모델)을 활용해 **상태 기반(state‑based) 다중 액터(multi‑actor) 애플리케이션**을 쉽고 효율적으로 구축할 수 있게 해 주는 오픈소스 라이브러리입니다.  
주요 목표는 다음과 같습니다.

| 목표 | 설명 |
|------|

In [97]:
graph.channels

{'messages': <langgraph.channels.binop.BinaryOperatorAggregate at 0x78d54bc74780>,
 '__start__': <langgraph.channels.ephemeral_value.EphemeralValue at 0x78d54bc74900>,
 '__pregel_tasks': <langgraph.channels.topic.Topic at 0x78d54bc75200>,
 'branch:to:chatbot': <langgraph.channels.ephemeral_value.EphemeralValue at 0x78d54bc753c0>,
 'branch:to:tools': <langgraph.channels.ephemeral_value.EphemeralValue at 0x78d54bc75d80>}

In [99]:
list(graph.channels.keys())

['messages',
 '__start__',
 '__pregel_tasks',
 'branch:to:chatbot',
 'branch:to:tools']

# Interrupt 후 메시지 상태 업데이트

In [75]:
from random import random

config = RunnableConfig(recursion_limit=10, configurable={"thread_id": random()})

for event in graph.stream({"messages": [("user", "랭그래프에 대해서 알려줘")]}, config, interrupt_before=["tools"]):
  for key, value in event.items():
    print(f"\n[{key}]\n")
    if "messages" in value:
      value["messages"][-1].pretty_print()


[chatbot]

================================== Ai Message ==================================
Tool Calls:
  tavily_search (fc_c2c33e7c-ed2a-4b77-9c1f-23baa1de3169)
 Call ID: fc_c2c33e7c-ed2a-4b77-9c1f-23baa1de3169
  Args:
    query: 랭그래프 랭킹 그래프
    search_depth: advanced

[__interrupt__]



In [76]:
snapshot = graph.get_state(config)

In [77]:
snapshot

StateSnapshot(values={'messages': [HumanMessage(content='랭그래프에 대해서 알려줘', additional_kwargs={}, response_metadata={}, id='7feaf6b8-b4ea-45ae-807a-f7f00420b446'), AIMessage(content='', additional_kwargs={'reasoning_content': 'User asks in Korean: "랭그래프에 대해서 알려줘" meaning "Tell me about ranking graph" or "Tell me about \'랭그래프\'". Likely they mean "랭킹 그래프" or "랭크 그래프" which might be "Ranking Graph" maybe a type of graph used in ranking or ranking visualization. Might refer to "랭그래프" as a specific term in Korean. Could be "랭그 그래프" but spelled "랭그래프". Might be a "랭그" meaning "ranking" in Korean. They might want an explanation of what a ranking graph is, how to create one, typical uses, examples. Provide in Korean. No external references required. We can provide a thorough explanation.\n\nWe might need to search to confirm if "랭그래프" is a known term. Let\'s search.', 'tool_calls': [{'id': 'fc_c2c33e7c-ed2a-4b77-9c1f-23baa1de3169', 'function': {'arguments': '{"query":"랭그래프 랭킹 그래프","search_depth"

In [78]:
snapshot.next

('tools',)

In [79]:
snapshot.values["messages"][-1]

AIMessage(content='', additional_kwargs={'reasoning_content': 'User asks in Korean: "랭그래프에 대해서 알려줘" meaning "Tell me about ranking graph" or "Tell me about \'랭그래프\'". Likely they mean "랭킹 그래프" or "랭크 그래프" which might be "Ranking Graph" maybe a type of graph used in ranking or ranking visualization. Might refer to "랭그래프" as a specific term in Korean. Could be "랭그 그래프" but spelled "랭그래프". Might be a "랭그" meaning "ranking" in Korean. They might want an explanation of what a ranking graph is, how to create one, typical uses, examples. Provide in Korean. No external references required. We can provide a thorough explanation.\n\nWe might need to search to confirm if "랭그래프" is a known term. Let\'s search.', 'tool_calls': [{'id': 'fc_c2c33e7c-ed2a-4b77-9c1f-23baa1de3169', 'function': {'arguments': '{"query":"랭그래프 랭킹 그래프","search_depth":"advanced"}', 'name': 'tavily_search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 221, 'prompt_tokens': 1364, 'total_tokens

In [80]:
# 마지막 메시지 가져오기
last_message = snapshot.values["messages"][-1]
last_message.pretty_print()
# 메시지에서 툴콜링 데이터 가져오기
last_message.tool_calls

================================== Ai Message ==================================
Tool Calls:
  tavily_search (fc_c2c33e7c-ed2a-4b77-9c1f-23baa1de3169)
 Call ID: fc_c2c33e7c-ed2a-4b77-9c1f-23baa1de3169
  Args:
    query: 랭그래프 랭킹 그래프
    search_depth: advanced


[{'name': 'tavily_search',
  'args': {'query': '랭그래프 랭킹 그래프', 'search_depth': 'advanced'},
  'id': 'fc_c2c33e7c-ed2a-4b77-9c1f-23baa1de3169',
  'type': 'tool_call'}]

In [81]:
last_message.tool_calls[0]["args"] = {'query': 'langgraph site:https://www.reddit.com/', 'search_depth': 'advanced'}

In [82]:
last_message

AIMessage(content='', additional_kwargs={'reasoning_content': 'User asks in Korean: "랭그래프에 대해서 알려줘" meaning "Tell me about ranking graph" or "Tell me about \'랭그래프\'". Likely they mean "랭킹 그래프" or "랭크 그래프" which might be "Ranking Graph" maybe a type of graph used in ranking or ranking visualization. Might refer to "랭그래프" as a specific term in Korean. Could be "랭그 그래프" but spelled "랭그래프". Might be a "랭그" meaning "ranking" in Korean. They might want an explanation of what a ranking graph is, how to create one, typical uses, examples. Provide in Korean. No external references required. We can provide a thorough explanation.\n\nWe might need to search to confirm if "랭그래프" is a known term. Let\'s search.', 'tool_calls': [{'id': 'fc_c2c33e7c-ed2a-4b77-9c1f-23baa1de3169', 'function': {'arguments': '{"query":"랭그래프 랭킹 그래프","search_depth":"advanced"}', 'name': 'tavily_search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 221, 'prompt_tokens': 1364, 'total_tokens

In [83]:
from langchain_core.messages import AIMessage

new_message = AIMessage(
    content=last_message.content,
    tool_calls=last_message.tool_calls,
    id=last_message.id
)

In [84]:
new_message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  tavily_search (fc_c2c33e7c-ed2a-4b77-9c1f-23baa1de3169)
 Call ID: fc_c2c33e7c-ed2a-4b77-9c1f-23baa1de3169
  Args:
    query: langgraph site:https://www.reddit.com/
    search_depth: advanced


In [85]:
graph.update_state(config, {"messages": [new_message]})

{'configurable': {'thread_id': '0.4491208059321644',
  'checkpoint_ns': '',
  'checkpoint_id': '1f09886f-fa5e-6a2d-8002-f1e94be5e76a'}}

In [86]:
graph.get_state(config).next

('tools',)

In [87]:
for event in graph.stream(None, config, stream_mode="values"):
  for key, value in event.items():
    print(f"\n[ {key} ]\n")
    print(value[-1])


[ messages ]

content='' additional_kwargs={} response_metadata={} id='run--3ead6010-9c0d-4ed6-9967-43b88a3b3e7d-0' tool_calls=[{'name': 'tavily_search', 'args': {'query': 'langgraph site:https://www.reddit.com/', 'search_depth': 'advanced'}, 'id': 'fc_c2c33e7c-ed2a-4b77-9c1f-23baa1de3169', 'type': 'tool_call'}]

[ messages ]

content='{"query": "langgraph", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://medium.com/@ken_lin/langgraph-a-framework-for-building-stateful-multi-agent-llm-applications-a51d5eb68d03", "title": "LangGraph: A Framework for Building Stateful Multi-Agent LLM ...", "content": "> LangGraph is a powerful Python library designed for constructing stateful, multi-actor applications with Large Language Models (LLMs). It extends the capabilities of LangChain Expression Language (LCEL) while specifically addressing limitations in existing frameworks for agent development. As a specialized tool for creating complex LLM applications,

In [93]:
snapshot = graph.get_state(config)
snapshot

StateSnapshot(values={'messages': [HumanMessage(content='랭그래프에 대해서 알려줘', additional_kwargs={}, response_metadata={}, id='7feaf6b8-b4ea-45ae-807a-f7f00420b446'), AIMessage(content='', additional_kwargs={}, response_metadata={}, id='run--3ead6010-9c0d-4ed6-9967-43b88a3b3e7d-0', tool_calls=[{'name': 'tavily_search', 'args': {'query': 'langgraph site:https://www.reddit.com/', 'search_depth': 'advanced'}, 'id': 'fc_c2c33e7c-ed2a-4b77-9c1f-23baa1de3169', 'type': 'tool_call'}]), ToolMessage(content='{"query": "langgraph", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://medium.com/@ken_lin/langgraph-a-framework-for-building-stateful-multi-agent-llm-applications-a51d5eb68d03", "title": "LangGraph: A Framework for Building Stateful Multi-Agent LLM ...", "content": "> LangGraph is a powerful Python library designed for constructing stateful, multi-actor applications with Large Language Models (LLMs). It extends the capabilities of LangChain Expression Langu

In [94]:
dir(snapshot)

['__add__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__match_args__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '_asdict',
 '_field_defaults',
 '_fields',
 '_make',
 '_replace',
 'config',
 'count',
 'created_at',
 'index',
 'interrupts',
 'metadata',
 'next',
 'parent_config',
 'tasks',
 'values']

In [92]:
snapshot.values["messages"]

[HumanMessage(content='랭그래프에 대해서 알려줘', additional_kwargs={}, response_metadata={}, id='7feaf6b8-b4ea-45ae-807a-f7f00420b446'),
 AIMessage(content='', additional_kwargs={}, response_metadata={}, id='run--3ead6010-9c0d-4ed6-9967-43b88a3b3e7d-0', tool_calls=[{'name': 'tavily_search', 'args': {'query': 'langgraph site:https://www.reddit.com/', 'search_depth': 'advanced'}, 'id': 'fc_c2c33e7c-ed2a-4b77-9c1f-23baa1de3169', 'type': 'tool_call'}]),
 ToolMessage(content='{"query": "langgraph", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://medium.com/@ken_lin/langgraph-a-framework-for-building-stateful-multi-agent-llm-applications-a51d5eb68d03", "title": "LangGraph: A Framework for Building Stateful Multi-Agent LLM ...", "content": "> LangGraph is a powerful Python library designed for constructing stateful, multi-actor applications with Large Language Models (LLMs). It extends the capabilities of LangChain Expression Language (LCEL) while specifically ad

In [105]:
config = RunnableConfig(recursion_limit=10, configurable={"thread_id": random()})

input = {"messages": [("user", "랭그래프에 대해서 알려줘")]}

for event in graph.stream(input, config, stream_mode="values"):
  for key, value in event.items():
    value[-1].pretty_print()

================================ Human Message =================================

랭그래프에 대해서 알려줘
================================== Ai Message ==================================
Tool Calls:
  tavily_search (fc_dac5cd07-186f-43a4-855a-a1d4f042dbda)
 Call ID: fc_dac5cd07-186f-43a4-855a-a1d4f042dbda
  Args:
    end_date: None
    exclude_domains: []
    include_domains: []
    include_favicon: False
    include_images: False
    query: 랭그래프 랭크 그래프
    search_depth: advanced
    start_date: None
    time_range: None
    topic: general
================================= Tool Message =================================
Name: tavily_search

{"query": "랭그래프 랭크 그래프", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://brunch.co.kr/@zer0cero/132", "title": "23화 22. LangGraph 기본과 응용 - 브런치", "content": "LangGraph(랭그래프)는 에이전트와 도구의 처리를 노드로 표현하고 노드 간의 연결을 엣지로 표현합니다. 가장자리는 다음에 실행할 노드를 지정합니다.", "score": 0.67706895, "raw_content": null}, {"url": "https://www.youtube.com/w

# 지난 스냅샷의 결과 수정하고 Replay 하기

In [278]:
# 지난 스냅샷의 상태를 출력
for state in graph.get_state_history(config):
  # print(state)
  print(state.config)
  print(state.config['configurable']['checkpoint_id'])

  messages = state.values["messages"]
  if len(messages) > 0:
    print(messages[-1].id)
    messages[-1].pretty_print()

  print("="*80)

{'configurable': {'thread_id': '0.035412983531594056', 'checkpoint_ns': '', 'checkpoint_id': '1f09890d-8639-6956-8006-827d86476864'}}
1f09890d-8639-6956-8006-827d86476864
run--b4968f8d-4880-426f-b7f1-680d96207210-0
================================== Ai Message ==================================

## LangGraph – 한 줄로 정리

> **LangGraph**는 LangChain 에코시스템 안에서 동작하는 **그래프 기반** 오케스트레이션 프레임워크입니다.  
> LLM(대형 언어 모델) 기반 에이전트들을 “노드”(Agent/Chain)로 만들고, 이 노드들을 **사이클(루프)** 을 포함한 **그래프** 형태로 연결해 복잡한 워크플로우를 구현할 수 있습니다.  
> <https://langchain-ai.github.io/langgraph/>

---

## 핵심 개념

| 개념 | 설명 | 예시 |
|------|------|------|
| **그래프 (Graph)** | 에이전트와 체인(작업)을 정점으로, 흐름을 간선으로 표현. DAG(Directed Acyclic Graph) 뿐 아니라 **순환** 구조도 지원. | FAQ 챗봇 → 정보 수집 → 요약 → 사용자 피드백 → 다시 수집 |
| **노드 (Node)** | LLM 호출, 외부 API 호출, 데이터 처리 등 하나의 단일 작업. | `ChatPromptTemplate`, `Tool`, `LLMChain` |
| **스테이트 (State)** | 각 실행 단계에서 보존되는 컨텍스트(예: 대화 기록, 메모리, 변수). | `AgentState`, `GraphState` |
| **리플렉션 (Reflection)** | 이전 실행 결과를 평가하고, 다음 행동을 결

In [279]:
state_history = graph.get_state_history(
    config,
    before=RunnableConfig(configurable={"checkpoint_id":"1f098899-6112-698f-8002-d72a3737d977"}),
    limit=1
)
to_reply_state = list(state_history)[0]
to_reply_state.values["messages"][-1].pretty_print()

================================== Ai Message ==================================
Tool Calls:
  tavily_search (fc_dac5cd07-186f-43a4-855a-a1d4f042dbda)
 Call ID: fc_dac5cd07-186f-43a4-855a-a1d4f042dbda
  Args:
    end_date: None
    exclude_domains: []
    include_domains: []
    include_favicon: False
    include_images: False
    query: 랭그래프 랭크 그래프
    search_depth: advanced
    start_date: None
    time_range: None
    topic: general


In [280]:
message = to_reply_state.values["messages"][-1]
message

AIMessage(content='', additional_kwargs={'reasoning_content': 'User wants info about 랭그래프. Likely Korean term 랭그래프 might be "ranking graph" or "ranking chart". Might refer to "랭킹 그래프" used in Korean for ranking of items over time, e.g., "랭그래프" might be a term for "ranking graph" used in some contexts. Could be a Korean term for "ranking graph" in data visualization. Let\'s search.', 'tool_calls': [{'id': 'fc_dac5cd07-186f-43a4-855a-a1d4f042dbda', 'function': {'arguments': '{"end_date":null,"exclude_domains":[],"include_domains":[],"include_favicon":false,"include_images":false,"query":"랭그래프 랭크 그래프","search_depth":"advanced","start_date":null,"time_range":null,"topic":"general"}', 'name': 'tavily_search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 165, 'prompt_tokens': 1364, 'total_tokens': 1529, 'completion_time': 0.147199626, 'prompt_time': 0.12519494, 'queue_time': 0.002709977, 'total_time': 0.272394566}, 'model_name': 'openai/gpt-oss-20b', 'syste

In [281]:
tool_calls = message.tool_calls.copy()
tool_calls

[{'name': 'tavily_search',
  'args': {'end_date': None,
   'exclude_domains': [],
   'include_domains': [],
   'include_favicon': False,
   'include_images': False,
   'query': '랭그래프 랭크 그래프',
   'search_depth': 'advanced',
   'start_date': None,
   'time_range': None,
   'topic': 'general'},
  'id': 'fc_dac5cd07-186f-43a4-855a-a1d4f042dbda',
  'type': 'tool_call'}]

In [282]:
tool_calls[0]['args'] = {
    **tool_calls[0]['args'],
    'query': 'langgraph site:reddit.com',
}

In [283]:
tool_calls

[{'name': 'tavily_search',
  'args': {'end_date': None,
   'exclude_domains': [],
   'include_domains': [],
   'include_favicon': False,
   'include_images': False,
   'query': 'langgraph site:reddit.com',
   'search_depth': 'advanced',
   'start_date': None,
   'time_range': None,
   'topic': 'general'},
  'id': 'fc_dac5cd07-186f-43a4-855a-a1d4f042dbda',
  'type': 'tool_call'}]

In [284]:
new_message = AIMessage(
    id=message.id, # 중요
    content=message.content,
    tool_calls=tool_calls
)
new_message

AIMessage(content='', additional_kwargs={}, response_metadata={}, id='run--e6d5142d-189e-41e0-b466-c3519f86a1be-0', tool_calls=[{'name': 'tavily_search', 'args': {'end_date': None, 'exclude_domains': [], 'include_domains': [], 'include_favicon': False, 'include_images': False, 'query': 'langgraph site:reddit.com', 'search_depth': 'advanced', 'start_date': None, 'time_range': None, 'topic': 'general'}, 'id': 'fc_dac5cd07-186f-43a4-855a-a1d4f042dbda', 'type': 'tool_call'}])

In [285]:
to_reply_state.config

{'configurable': {'thread_id': '0.035412983531594056',
  'checkpoint_ns': '',
  'checkpoint_id': '1f098899-38bd-69f8-8001-62abd3d26a33'}}

In [286]:
config

{'recursion_limit': 10, 'configurable': {'thread_id': 0.035412983531594056}}

In [287]:
to_reply_state.config

{'configurable': {'thread_id': '0.035412983531594056',
  'checkpoint_ns': '',
  'checkpoint_id': '1f098899-38bd-69f8-8001-62abd3d26a33'}}

In [288]:
updated_state = graph.update_state(config, {"messages": [new_message]})
updated_state

{'configurable': {'thread_id': '0.035412983531594056',
  'checkpoint_ns': '',
  'checkpoint_id': '1f09890e-ecd1-66f9-8007-ad99be1670b4'}}

In [289]:
for event in graph.stream(None, to_reply_state.config, stream_mode="values"):
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================== Ai Message ==================================
Tool Calls:
  tavily_search (fc_dac5cd07-186f-43a4-855a-a1d4f042dbda)
 Call ID: fc_dac5cd07-186f-43a4-855a-a1d4f042dbda
  Args:
    end_date: None
    exclude_domains: []
    include_domains: []
    include_favicon: False
    include_images: False
    query: 랭그래프 랭크 그래프
    search_depth: advanced
    start_date: None
    time_range: None
    topic: general
================================= Tool Message =================================
Name: tavily_search

{"query": "랭그래프 랭크 그래프", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://brunch.co.kr/@zer0cero/132", "title": "23화 22. LangGraph 기본과 응용 - 브런치", "content": "LangGraph(랭그래프)는 에이전트와 도구의 처리를 노드로 표현하고 노드 간의 연결을 엣지로 표현합니다. 가장자리는 다음에 실행할 노드를 지정합니다.", "score": 0.67706895, "raw_content": null}, {"url": "https://www.youtube.com/watch?v=W_uwR_yx4-c", "title": "#LangGraph 개념 완전 정복 몰아보기(3시간) - YouTube", "content": "랭체인을 쓰지 않아도

In [294]:
snapshot = graph.get_state(config)

In [295]:
snapshot

StateSnapshot(values={'messages': [HumanMessage(content='랭그래프에 대해서 알려줘', additional_kwargs={}, response_metadata={}, id='7b915ff2-f0a3-4852-9025-6d34600f4195'), AIMessage(content='', additional_kwargs={'reasoning_content': 'User wants info about 랭그래프. Likely Korean term 랭그래프 might be "ranking graph" or "ranking chart". Might refer to "랭킹 그래프" used in Korean for ranking of items over time, e.g., "랭그래프" might be a term for "ranking graph" used in some contexts. Could be a Korean term for "ranking graph" in data visualization. Let\'s search.', 'tool_calls': [{'id': 'fc_dac5cd07-186f-43a4-855a-a1d4f042dbda', 'function': {'arguments': '{"end_date":null,"exclude_domains":[],"include_domains":[],"include_favicon":false,"include_images":false,"query":"랭그래프 랭크 그래프","search_depth":"advanced","start_date":null,"time_range":null,"topic":"general"}', 'name': 'tavily_search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 165, 'prompt_tokens': 1364, 'total_tokens': 1

In [301]:
snapshot.values['messages']

[HumanMessage(content='랭그래프에 대해서 알려줘', additional_kwargs={}, response_metadata={}, id='7b915ff2-f0a3-4852-9025-6d34600f4195'),
 AIMessage(content='', additional_kwargs={'reasoning_content': 'User wants info about 랭그래프. Likely Korean term 랭그래프 might be "ranking graph" or "ranking chart". Might refer to "랭킹 그래프" used in Korean for ranking of items over time, e.g., "랭그래프" might be a term for "ranking graph" used in some contexts. Could be a Korean term for "ranking graph" in data visualization. Let\'s search.', 'tool_calls': [{'id': 'fc_dac5cd07-186f-43a4-855a-a1d4f042dbda', 'function': {'arguments': '{"end_date":null,"exclude_domains":[],"include_domains":[],"include_favicon":false,"include_images":false,"query":"랭그래프 랭크 그래프","search_depth":"advanced","start_date":null,"time_range":null,"topic":"general"}', 'name': 'tavily_search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 165, 'prompt_tokens': 1364, 'total_tokens': 1529, 'completion_time': 0.1471996

In [297]:
type(snapshot)

langgraph.types.StateSnapshot

In [298]:
dir(snapshot)

['__add__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__match_args__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '_asdict',
 '_field_defaults',
 '_fields',
 '_make',
 '_replace',
 'config',
 'count',
 'created_at',
 'index',
 'interrupts',
 'metadata',
 'next',
 'parent_config',
 'tasks',
 'values']

In [299]:
snapshot.config

{'configurable': {'thread_id': '0.035412983531594056',
  'checkpoint_ns': '',
  'checkpoint_id': '1f09890f-0f83-693b-8003-9bdbffe5b7d0'}}